In [1]:
#필요한 라이브러리 import
import requests
import urllib.request
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import pymysql

In [38]:
conn = pymysql.connect(host="localhost", user='queant', password='queant1234', db = 'queant', charset='utf8mb4')
cur = conn.cursor()

**의논사항**
* product table에서 은행 id Long으로 선언되어 있었는데 Long이 없는것 같아서 int로 변경
* 가입방법 table에 가입방법 column이 필요 없을 것 같다고 생각이 듦.

create product table
```sql
CREATE TABLE `queant`.`product` (
  `product_id` INT NOT NULL AUTO_INCREMENT,
  `bank_id` INT NOT NULL,
  `is_deposit` TINYINT NOT NULL,
  `name` VARCHAR(50) NOT NULL,
  PRIMARY KEY (`product_id`));
```

create joinway table
```sql
CREATE TABLE `queant`.`joinway` (
  `joinway_id` INT NOT NULL AUTO_INCREMENT,
  `product_id` INT NOT NULL,
  `code_id` VARCHAR(2) NOT NULL,
  `scode_id` INT NOT NULL,
  PRIMARY KEY (`joinway_id`));
```


In [3]:
#api활용할 url부분

#각 데이터 별 url 앞부분
url_deposit = ["https://finlife.fss.or.kr/finlifeapi/depositProductsSearch.xml?auth=", "https://finlife.fss.or.kr/finlifeapi/savingProductsSearch.xml?auth="]

#발급받은 키
api_key = "47c0e868fdb16333d47d0e385641c3c0"

#권역코드
code_front = "&topFinGrpNo="
code_bank = ["020000", "030300"]

#페이지
page_front = "&pageNo="
page_num = ["1", "2", "3", "4"]

In [4]:
is_deposit = 1
url = url_deposit[0] + api_key + code_front + code_bank[is_deposit] + page_front + page_num[0]
print(url)
data_str = urllib.request.urlopen(url).read().decode('euc-kr')
data_xml = ET.fromstring(data_str)

https://finlife.fss.or.kr/finlifeapi/depositProductsSearch.xml?auth=47c0e868fdb16333d47d0e385641c3c0&topFinGrpNo=030300&pageNo=1


In [43]:
#상품 table에 정보들을 담는다.
#python은 전부다 %s를 써야함.
query_prdt_search = """select * from queant.product where bank_id = (%s) and is_deposit = (%s) and name = (%s)""" #중복체크 확인 쿼리문
query_prdt = """INSERT INTO queant.product (bank_id, is_deposit, name) values (%s,%s,%s);""" #데이터 insert 쿼리문
for product_tag in data_xml[5]:
    bank_id = int(product_tag[0].find("fin_co_no").text) #은행코드
    deposit = is_deposit #예금인지 적금인지
    prdt_name = product_tag[0].find("fin_prdt_nm").text #상품명
    prdt_name = prdt_name.replace("\n", " ")
    values = (bank_id,deposit,prdt_name) # 각각의 value들을 values에 묶어서 전달한다.
    cur.execute(query_prdt_search, values)
    if cur.fetchone() == None:
        cur.execute(query_prdt, values)
conn.commit()

In [39]:
# 공통코드에서 가입방법의 코드를 가져온다.
cur.execute("SELECT code_id FROM queant.commoncode where code_value = \"가입 방법\"")

row = cur.fetchone()
if row != None:
    code_id = row[0] #A가 저장됨

In [41]:
#가입 방법 코드
query_find_common = """SELECT * FROM queant.specificcode where code_id = (%s)"""
cur.execute(query_find_common,code_id)

join_ways = {}
while True:
    row = cur.fetchone()
    if row == None:
        break
    join_ways[row[2]] = int(row[0])
    
print(join_ways)

{'인터넷': 1, '스마트폰': 2, '전화': 3, '기타': 4, '영업점': 5}


In [44]:
query_join = """INSERT INTO queant.joinway (product_id, code_id, scode_id) values (%s,%s,%s);"""
query_find_prdtid = """select product_id from queant.product where name = (%s) and bank_id = (%s)"""
query_find_join = "select * from queant.joinway where product_id = (%s) and code_id = (%s) and scode_id = (%s)"
for product_tag in data_xml[5]:
    prdt_name = product_tag[0].find("fin_prdt_nm").text #상품명
    bank_id = int(product_tag[0].find("fin_co_no").text) #은행코드
    values = (prdt_name, bank_id)
    cur.execute(query_find_prdtid, values)
    product_id = cur.fetchone()[0]
    join = product_tag[0].find("join_way").text
    for join_way in join_ways:
        if join_way in join:
            values = (product_id, code_id, join_ways[join_way])
            cur.execute(query_find_join, values)
            if cur.fetchone() == None:
                cur.execute(query_join, values)
conn.commit()

In [ ]:
cur.execute("select * from queant.joinway as jw left join queant.product as prdt on prdt.product_id = jw.product_id right join queant.specificcode as spec on jw.scode_id = spec.scode_id;")
while True:
    row = cur.fetchone()
    if row == None:
        break
    print(row)


In [45]:
conn.close()